# Final Model

## Import libraries & data

In [1]:
import gurobipy as gb
from gurobipy import *
import pandas as pd

In [2]:
xlsx = pd.ExcelFile('Cleaned1.xlsx')
df_orders = pd.read_excel(xlsx, 'OrderList')
df_rates = pd.read_excel(xlsx, 'Rates')
df_costs = pd.read_excel(xlsx, 'WhCosts')
df_capacities = pd.read_excel(xlsx, 'WhCapacities')
df_customers = pd.read_excel(xlsx, 'VmiCustomers')
df_ports = pd.read_excel(xlsx, 'PlantPorts')

In [3]:
df_ports

,Plant Code,Port
0,PLANT01,PORT01
1,PLANT01,PORT02
2,PLANT02,PORT03
3,PLANT03,PORT04
4,PLANT04,PORT05
5,PLANT05,PORT06
6,PLANT06,PORT06
7,PLANT07,PORT01
8,PLANT07,PORT02
9,PLANT08,PORT04


## Values for Variables

In [4]:
n_orders = df_orders.shape[0]                                   # Number of Orders
n_ports = int(df_rates['orig_port_cd'].value_counts().count())  # Number of Ports
n_plants = int(df_costs['WH'].value_counts().count())           # Number of Plants

In [6]:
n_orders, n_ports, n_plants

(9215, 10, 19)

## Values for Objective Function

In [8]:
costs_per_weight = list(df_rates.sort_values(by = ['orig_port_cd'])['rate'])
minimum_costs = list(df_rates.sort_values(by = ['orig_port_cd'])['minimum cost'])

unit_to_transport = df_orders['Unit quantity'].values.tolist()
weight_to_transport = df_orders['Weight'].values.tolist()

costs_per_unit = list(df_costs.sort_values(by = ['WH'])['Cost/unit'])
daily_capacity = list(df_capacities.sort_values(by = ['Plant ID'])['Capacity '])

## Plant-to-Port Constraint

In [13]:
port_to_plant = {}

for i in range(len(df_ports)):
    if i != 0:
        port_to_plant[str(int(df_ports.loc[i]['Port'][-2:]))] = []
for i in range(len(df_ports)):
    if i != 0:
        port_to_plant[str(int(df_ports.loc[i]['Port'][-2:]))].append(int(df_ports.loc[i]['Plant Code'][-2:]) - 1)

port_to_plant_matrix = {}
for i in range(n_ports):
    port_to_plant_matrix[str(i + 2)] = [0] * n_plants
    
for port_key in port_to_plant_matrix:
    for plant_key in port_to_plant[port_key]:
        port_to_plant_matrix[port_key][plant_key] = 1
 

In [12]:
       
port_to_plant_matrix = pd.DataFrame(port_to_plant_matrix)
port_to_plant_matrix = port_to_plant_matrix.transpose().values.tolist()
port_to_plant_matrix

[[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

In [14]:
port_to_plant_matrix

{'2': [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 '3': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 '4': [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1],
 '5': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 '6': [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 '7': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 '8': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 '9': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 '10': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 '11': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]}

## Customers Constraint

In [16]:
# Create customer mapping to assign each customer an integer code
i = 0
customer_mapping = {}
for customer in list(df_orders['Customer'].unique()):
    customer_mapping[customer] = i
    i += 1
n_customers = len(customer_mapping)

In [7]:
# Create matrix to record which customer made which order
customers = []
for i in range(len(df_orders)):
    customers.append(df_orders.loc[i]['Customer'])
    
for i in range(len(customers)):
    customer_id = customers[i]
    customers[i] = customer_mapping[customer_id]

customers_df = {}
for i in range(n_customers):
    customers_df[str(i)] = [0] * n_orders
customers_df = pd.DataFrame(customers_df)

for i in range(n_orders):
    customer_id = str(customers[i])
    customers_df[customer_id][i] = 1
    
customers_matrix = customers_df.values.tolist()
customers_df

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9213,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Change values in the original customers DataFrame to reflect new codes
df_customers_2 = df_customers
for i in range(len(df_customers_2)):
    customer = df_customers_2.loc[i]['Customers']
    if customer in customer_mapping:
        df_customers_2.loc[i]['Customers'] = customer_mapping[customer]
    
for i in range(len(df_customers_2)):
    if type(df_customers_2.loc[i]['Customers']) == str:
        df_customers_2 = df_customers_2.drop([i])

df_customers_2 = df_customers_2.reset_index(drop = True)

for i in range(len(df_customers_2)):
    df_customers_2['Plant Code'][i] = int(df_customers_2['Plant Code'][i][-2:]) - 1

df_customers_2

,Plant Code,Customers
0,1,37
1,1,1
2,1,12
3,1,33
4,1,34
5,1,35
6,1,13
7,5,35
8,9,1
9,9,18


In [19]:
# Record the restrictions above as a matrix
customer_to_plant_df = {}
for i in range(n_plants):
    customer_to_plant_df[str(i)] = [0] * n_customers
customer_to_plant_df = pd.DataFrame(customer_to_plant_df)

for i in range(len(df_customers_2)):
    plant = str(df_customers_2['Plant Code'][i])
    customer_id = df_customers_2['Customers'][i]
    customer_to_plant_df[plant][customer_id] = 1   

# There are some customers who do not care about which plant their orders are processed in
for i in range(n_customers):
    if sum(customer_to_plant_df.iloc[i]) == 0:
        for plant in range(n_plants):
            customer_to_plant_df[str(plant)][i] = 1

customer_to_plant = customer_to_plant_df.values.tolist()
customer_to_plant_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


## Solving the Problem

In [20]:
prob = gb.Model('Supply Chain Allocation')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-10


In [21]:
# X is a matrix that records which orders goes to which port
X = prob.addVars(n_orders, n_ports, vtype = GRB.BINARY, name = ['Order' + str(i + 1) + '_PORT' + str(j + 2) for i in range(n_orders) for j in range(n_ports)])              
# Y is a matrix that records which orders goes to which plant
Y = prob.addVars(n_orders, n_plants, vtype = GRB.BINARY, name = ['Order' + str(i + 1) + '_PLANT' + str(j + 1) for i in range(n_orders) for j in range(n_plants)])  
# W and A is the indicator of whether each port have at least 1 order for charging the starting fee
W = prob.addVars(n_ports, vtype = GRB.BINARY, name = ['Indicator of Order in port' + str(i + 2) for i in range(n_ports)])
A = prob.addVars(n_ports, vtype = GRB.BINARY)

In [13]:
# Total shipping cost (Plz ask Riley for the details)
total_cost = sum(minimum_costs[k]*W[k] for k in range(n_ports)) + sum(weight_to_transport[i] * X[i,k] * costs_per_weight[k]
                                 for i in range(n_orders) for k in range(n_ports)) + sum(unit_to_transport[i] * Y[i,j] * costs_per_unit[j] for i in range(n_orders) for j in range(n_plants))

# Since this is a minimization problem, we need to impose a penalty so that it fulfill as many orders
# as possible. If we do not do this, then no orders will be fulfilled
penalty = sum(1 - sum(X[i,j] for j in range(n_ports)) for i in range(n_orders)) * 100000

prob.setObjective(total_cost + penalty, GRB.MINIMIZE)

In [14]:
# Each fulfilled order will pass by 1 port and 1 plant
for i in range(n_orders):
    prob.addConstr(sum(X[i, j] for j in range(n_ports)) <= 1)
    prob.addConstr(sum(Y[i, j] for j in range(n_plants)) <= 1)

In [15]:
# If the order is sent to a port, then it must also be sent to a plant
# In theory we do not need this constraint (for reasons I will specify in the report)
# It's just here for precautions
for i in range(n_orders):
    prob.addConstr(sum(X[i,j] for j in range(n_ports)) == sum(Y[i,j] for j in range(n_plants)))

In [16]:
# The total order capacity cannot exceed the capacity of the plant
for j in range(n_plants):
    prob.addConstr(sum(unit_to_transport[i] * Y[i,j] for i in range(n_orders)) <= daily_capacity[j])

In [17]:
# Plant-to-port Constraint (matrix multiplication which I will explain in the report)
for i in range(n_orders):
    for k in range(n_plants):
        prob.addConstr(sum(X[i,j] * port_to_plant_matrix[j][k] for j in range(n_ports)) >= Y[i, k])

In [18]:
# Customer-to-plant Constraint (matrix multiplication which I will explain in the report)
for i in range(n_orders):
    for k in range(n_plants):
        prob.addConstr(sum(customers_matrix[i][j] * customer_to_plant[j][k] for j in range(n_customers)) >= Y[i, k])

In [19]:
# If Constraint for port starting fee
eps = 0.0001
M = 10 + eps 
for i in range(n_ports):
    prob.addConstr(A[i] == sum(X[j,i] for j in range(n_orders)))
    prob.addConstr(A[i] >= 0.5 + eps - M * (1 - W[i]))
    prob.addConstr(A[i] <= 0.5 + M * W[i])

In [20]:
prob.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 377864 rows, 267255 columns and 1327010 nonzeros
Model fingerprint: 0xc12f339c
Variable types: 0 continuous, 267255 integer (267255 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [7e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+03]
Found heuristic solution: objective 9.215000e+08
Presolve removed 335002 rows and 190556 columns (presolve time = 14s) ...
Presolve removed 364645 rows and 229788 columns
Presolve time: 13.79s
Presolved: 13219 rows, 37467 columns, 125709 nonzeros
Variable types: 0 continuous, 37467 integer (37467 binary)
Found heuristic solution: objective 9.209012e+08

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     

## Results

In [21]:
for v in prob.getVars():
    if v.x == 1:
        print(v.varName, ':', v.x)

Order1785_PORT9 : 1.0
Order2026_PORT2 : 1.0
Order4877_PORT5 : 1.0
Order5229_PORT6 : 1.0
Order6762_PORT7 : 1.0
Order7526_PORT4 : 1.0
Order1785_PLANT16 : 1.0
Order2026_PLANT7 : 1.0
Order4877_PLANT4 : 1.0
Order5229_PLANT5 : 1.0
Order6762_PLANT14 : 1.0
Order7526_PLANT13 : 1.0
Indicator of Order in port2 : 1.0
Indicator of Order in port4 : 1.0
Indicator of Order in port5 : 1.0
Indicator of Order in port6 : 1.0
Indicator of Order in port7 : 1.0
Indicator of Order in port9 : 1.0
C267245 : 1.0
C267247 : 1.0
C267248 : 1.0
C267249 : 1.0
C267250 : 1.0
C267252 : 1.0
